In [32]:
from torch.utils.data import Subset

In [1]:
import os
from PIL import Image
import torch
from torchvision.datasets import CocoDetection
from torchvision.transforms import functional as F

class COCODetectionRetinaNet(CocoDetection):
    def __init__(self, img_folder, ann_file, transforms=None, max_images=None):
        super().__init__(img_folder, ann_file)
        self._transforms = transforms

        # 🔥 FILTRA solo le immagini che esistono veramente su disco
        valid_ids = []
        for img_id in self.ids:
            info = self.coco.loadImgs(img_id)[0]
            fname = os.path.basename(info["file_name"])
            full_path = os.path.join(self.root, fname)
            if os.path.exists(full_path):
                valid_ids.append(img_id)

        # Se vuoi ancora ridurre il dataset, puoi limitare il numero
        if max_images is not None:
            valid_ids = valid_ids[:max_images]

        self.ids = valid_ids
        print(f"Using {len(self.ids)} valid images out of original {len(self.coco.getImgIds())}")

    def _load_image(self, id):
        img_info = self.coco.loadImgs(id)[0]
        path = img_info["file_name"]
        path = os.path.basename(path)  # ignoriamo eventuali sottocartelle strane
        return Image.open(os.path.join(self.root, path)).convert("RGB")

    def __getitem__(self, idx):
        img, targets = super().__getitem__(idx)

        boxes = []
        labels = []
        areas = []
        iscrowd = []

        for obj in targets:
            if obj.get("iscrowd", 0) == 1:
                continue
            x, y, w, h = obj["bbox"]
            x1 = x
            y1 = y
            x2 = x + w
            y2 = y + h
            boxes.append([x1, y1, x2, y2])
            labels.append(obj["category_id"])
            areas.append(obj["area"])
            iscrowd.append(obj.get("iscrowd", 0))

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        areas = torch.as_tensor(areas, dtype=torch.float32)
        iscrowd = torch.as_tensor(iscrowd, dtype=torch.int64)

        target = {
            "boxes": boxes,
            "labels": labels,
            "area": areas,
            "iscrowd": iscrowd,
            "image_id": torch.tensor([self.ids[idx]])
        }

        if self._transforms is not None:
            img, target = self._transforms(img, target)

        return img, target

In [2]:
# anchors.py
import math
import torch

class AnchorGenerator:
    """
    Generate anchors for each FPN level.
    sizes: list of base sizes per level, e.g. [32, 64, 128, 256, 512]
    aspect_ratios: list, es. [0.5, 1.0, 2.0]
    scales: list of scale factors per level, es. [1.0, 2 ** (1/3), 2 ** (2/3)]
    """
    def __init__(self, sizes, aspect_ratios, scales):
        self.sizes = sizes
        self.aspect_ratios = aspect_ratios
        self.scales = scales

    def grid_anchors(self, grid_size, stride, base_size):
        # grid_size: (h, w), stride: float
        device = grid_size.device if isinstance(grid_size, torch.Tensor) else torch.device("cpu")
        h, w = int(grid_size[0]), int(grid_size[1])
        shifts_x = torch.arange(0, w * stride, step=stride, device=device)
        shifts_y = torch.arange(0, h * stride, step=stride, device=device)
        shift_y, shift_x = torch.meshgrid(shifts_y, shifts_x, indexing="ij")
        shift_x = shift_x.reshape(-1)
        shift_y = shift_y.reshape(-1)

        anchors = []
        for scale in self.scales:
            for ar in self.aspect_ratios:
                area = (base_size * scale) ** 2.0
                w_anchor = math.sqrt(area / ar)
                h_anchor = ar * w_anchor
                x1 = shift_x - 0.5 * w_anchor
                y1 = shift_y - 0.5 * h_anchor
                x2 = shift_x + 0.5 * w_anchor
                y2 = shift_y + 0.5 * h_anchor
                anchors_per = torch.stack([x1, y1, x2, y2], dim=1)
                anchors.append(anchors_per)

        anchors = torch.cat(anchors, dim=0)
        return anchors

    def __call__(self, feature_maps, strides):
        """
        feature_maps: list of feature maps (tensor BxCxHxW)
        strides: list of strides for each level (e.g. [8, 16, 32, 64, 128])
        """
        all_anchors = []
        for level, (fm, stride, size) in enumerate(zip(feature_maps, strides, self.sizes)):
            _, _, h, w = fm.shape
            anchors = self.grid_anchors((h, w), stride, size)
            all_anchors.append(anchors)
        return all_anchors

In [14]:
# losses.py
import torch
import torch.nn.functional as F

def sigmoid_focal_loss(inputs, targets, alpha=0.25, gamma=2.0, reduction="sum"):
    """
    inputs: logits (N, num_classes)
    targets: one-hot or {0,1} (N, num_classes)
    """
    prob = torch.sigmoid(inputs)
    ce_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction="none")
    p_t = prob * targets + (1 - prob) * (1 - targets)
    loss = ce_loss * ((1 - p_t) ** gamma)

    if alpha >= 0:
        alpha_t = alpha * targets + (1 - alpha) * (1 - targets)
        loss = alpha_t * loss

    if reduction == "mean":
        return loss.mean()
    elif reduction == "sum":
        return loss.sum()
    return loss


def smooth_l1_loss(input, target, beta=1.0 / 9, reduction="sum"):
    """
    Smooth L1 as in Fast R-CNN.
    """
    n = torch.abs(input - target)
    cond = n < beta
    loss = torch.where(cond, 0.5 * n ** 2 / beta, n - 0.5 * beta)
    if reduction == "mean":
        return loss.mean()
    elif reduction == "sum":
        return loss.sum()
    return loss

In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import resnet50, ResNet50_Weights

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import resnet50, ResNet50_Weights


# ---------- LOSSES ----------

def sigmoid_focal_loss(inputs, targets, alpha=0.25, gamma=2.0, reduction="sum"):
    """
    inputs: logits (N, num_classes)
    targets: one-hot o {0,1} (N, num_classes)
    """
    prob = torch.sigmoid(inputs)
    ce_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction="none")
    p_t = prob * targets + (1 - prob) * (1 - targets)
    loss = ce_loss * ((1 - p_t) ** gamma)

    if alpha >= 0:
        alpha_t = alpha * targets + (1 - alpha) * (1 - targets)
        loss = alpha_t * loss

    if reduction == "mean":
        return loss.mean()
    elif reduction == "sum":
        return loss.sum()
    return loss


def smooth_l1_loss(input, target, beta=1.0 / 9, reduction="sum"):
    """
    Smooth L1 come in Fast R-CNN.
    """
    n = torch.abs(input - target)
    cond = n < beta
    loss = torch.where(cond, 0.5 * n ** 2 / beta, n - 0.5 * beta)
    if reduction == "mean":
        return loss.mean()
    elif reduction == "sum":
        return loss.sum()
    return loss


# ---------- Head (classification + regression) ----------

class RetinaNetHead(nn.Module):
    def __init__(self, in_channels, num_anchors, num_classes):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, in_channels, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels, in_channels, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels, in_channels, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels, in_channels, 3, padding=1),
            nn.ReLU(),
        )
        self.cls_logits = nn.Conv2d(in_channels, num_anchors * num_classes, 3, padding=1)
        self.bbox_pred = nn.Conv2d(in_channels, num_anchors * 4, 3, padding=1)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.normal_(m.weight, std=0.01)
                nn.init.constant_(m.bias, 0)

        # init del bias per cls (p ~ 0.01)
        pi = 0.01
        nn.init.constant_(self.cls_logits.bias, -torch.log(torch.tensor((1 - pi) / pi)))

        self.num_classes = num_classes
        self.num_anchors = num_anchors

    def forward(self, xs):
        logits = []
        bbox_reg = []
        for x in xs:
            y = self.conv(x)
            logits.append(self.cls_logits(y))
            bbox_reg.append(self.bbox_pred(y))
        return logits, bbox_reg


# ---------- FPN ----------

class FPN(nn.Module):
    def __init__(self, c3_channels, c4_channels, c5_channels, out_channels=256):
        super().__init__()
        self.lateral3 = nn.Conv2d(c3_channels, out_channels, 1)
        self.lateral4 = nn.Conv2d(c4_channels, out_channels, 1)
        self.lateral5 = nn.Conv2d(c5_channels, out_channels, 1)

        self.out3 = nn.Conv2d(out_channels, out_channels, 3, padding=1)
        self.out4 = nn.Conv2d(out_channels, out_channels, 3, padding=1)
        self.out5 = nn.Conv2d(out_channels, out_channels, 3, padding=1)

        self.p6 = nn.Conv2d(c5_channels, out_channels, 3, stride=2, padding=1)
        self.p7 = nn.Conv2d(out_channels, out_channels, 3, stride=2, padding=1)

    def forward(self, c3, c4, c5):
        # top-down pathway
        p5 = self.lateral5(c5)

        # 👇 upsample esplicitando la size del livello corrispondente
        p4 = self.lateral4(c4) + F.interpolate(p5, size=c4.shape[-2:], mode="nearest")
        p3 = self.lateral3(c3) + F.interpolate(p4, size=c3.shape[-2:], mode="nearest")

        p3 = self.out3(p3)
        p4 = self.out4(p4)
        p5 = self.out5(p5)

        p6 = self.p6(c5)
        p7 = self.p7(F.relu(p6))

        return [p3, p4, p5, p6, p7]


# ---------- Anchor generator ----------

class AnchorGenerator:
    def __init__(self, sizes, aspect_ratios, scales):
        self.sizes = sizes
        self.aspect_ratios = aspect_ratios
        self.scales = scales

    def grid_anchors(self, grid_size, stride, base_size, device):
        h, w = int(grid_size[0]), int(grid_size[1])
        shifts_x = torch.arange(0, w * stride, step=stride, device=device)
        shifts_y = torch.arange(0, h * stride, step=stride, device=device)
        shift_y, shift_x = torch.meshgrid(shifts_y, shifts_x, indexing="ij")
        shift_x = shift_x.reshape(-1)
        shift_y = shift_y.reshape(-1)

        anchors = []
        for scale in self.scales:
            for ar in self.aspect_ratios:
                area = (base_size * scale) ** 2.0
                w_anchor = torch.sqrt(torch.tensor(area / ar, device=device))
                h_anchor = ar * w_anchor
                x1 = shift_x - 0.5 * w_anchor
                y1 = shift_y - 0.5 * h_anchor
                x2 = shift_x + 0.5 * w_anchor
                y2 = shift_y + 0.5 * h_anchor
                anchors_per = torch.stack([x1, y1, x2, y2], dim=1)
                anchors.append(anchors_per)

        anchors = torch.cat(anchors, dim=0)
        return anchors

    def __call__(self, feature_maps, strides, device):
        all_anchors = []
        for fm, stride, size in zip(feature_maps, strides, self.sizes):
            _, _, h, w = fm.shape
            anchors = self.grid_anchors((h, w), stride, size, device)
            all_anchors.append(anchors)
        return all_anchors


# ---------- Utility IoU / encoding ----------

def box_iou(boxes1, boxes2):
    area1 = (boxes1[:, 2] - boxes1[:, 0]).clamp(min=0) * (boxes1[:, 3] - boxes1[:, 1]).clamp(min=0)
    area2 = (boxes2[:, 2] - boxes2[:, 0]).clamp(min=0) * (boxes2[:, 3] - boxes2[:, 1]).clamp(min=0)

    lt = torch.max(boxes1[:, None, :2], boxes2[:, :2])  # (N,M,2)
    rb = torch.min(boxes1[:, None, 2:], boxes2[:, 2:])

    wh = (rb - lt).clamp(min=0)
    inter = wh[:, :, 0] * wh[:, :, 1]
    union = area1[:, None] + area2 - inter
    iou = inter / union.clamp(min=1e-6)
    return iou


def encode_boxes(anchors, gt_boxes):
    aw = anchors[:, 2] - anchors[:, 0]
    ah = anchors[:, 3] - anchors[:, 1]
    ax = anchors[:, 0] + 0.5 * aw
    ay = anchors[:, 1] + 0.5 * ah

    gw = gt_boxes[:, 2] - gt_boxes[:, 0]
    gh = gt_boxes[:, 3] - gt_boxes[:, 1]
    gx = gt_boxes[:, 0] + 0.5 * gw
    gy = gt_boxes[:, 1] + 0.5 * gh

    tx = (gx - ax) / aw
    ty = (gy - ay) / ah
    tw = torch.log(gw / aw)
    th = torch.log(gh / ah)

    return torch.stack([tx, ty, tw, th], dim=1)


# ---------- RetinaNet completo ----------

class RetinaNet(nn.Module):
    def __init__(self, num_classes, backbone_pretrained=True):
        super().__init__()

        # Backbone ResNet-50
        weights = ResNet50_Weights.DEFAULT if backbone_pretrained else None
        backbone = resnet50(weights=weights)
        self.backbone = backbone

        # canali C3, C4, C5
        c3_channels = backbone.layer2[-1].conv3.out_channels
        c4_channels = backbone.layer3[-1].conv3.out_channels
        c5_channels = backbone.layer4[-1].conv3.out_channels

        self.fpn = FPN(c3_channels, c4_channels, c5_channels, out_channels=256)

        # Anchors
        self.anchor_generator = AnchorGenerator(
            sizes=[32, 64, 128, 256, 512],
            aspect_ratios=[0.5, 1.0, 2.0],
            scales=[1.0, 2 ** (1/3), 2 ** (2/3)]
        )
        num_anchors = len(self.anchor_generator.aspect_ratios) * len(self.anchor_generator.scales)
        self.head = RetinaNetHead(256, num_anchors, num_classes)

        self.strides = [8, 16, 32, 64, 128]

    def extract_backbone_features(self, x):
        # forward ResNet standard
        x = self.backbone.conv1(x)
        x = self.backbone.bn1(x)
        x = self.backbone.relu(x)
        x = self.backbone.maxpool(x)

        c2 = self.backbone.layer1(x)  # /4
        c3 = self.backbone.layer2(c2) # /8
        c4 = self.backbone.layer3(c3) # /16
        c5 = self.backbone.layer4(c4) # /32

        return c3, c4, c5

    def forward(self, images, targets=None):
        # images: list di tensor [3,H,W] già pad-dati nel collate_fn
        device = images[0].device
        imgs = torch.stack(images, dim=0)   # (B,3,H,W)

        c3, c4, c5 = self.extract_backbone_features(imgs)
        features = self.fpn(c3, c4, c5)
        cls_logits, bbox_reg = self.head(features)

        # reshape
        batch_size = imgs.size(0)
        all_anchors = self.anchor_generator(features, self.strides, device)
        anchors = torch.cat(all_anchors, dim=0)  # (A_total, 4)

        cls_list = []
        box_list = []
        for l, b in zip(cls_logits, bbox_reg):
            N, AxC, H, W = l.shape
            Ax4 = b.shape[1]
            A = Ax4 // 4
            C = AxC // A

            l = l.permute(0, 2, 3, 1).reshape(N, -1, C)
            b = b.permute(0, 2, 3, 1).reshape(N, -1, 4)
            cls_list.append(l)
            box_list.append(b)

        cls_out = torch.cat(cls_list, dim=1)   # (B, A_total, C)
        box_out = torch.cat(box_list, dim=1)   # (B, A_total, 4)

        if self.training:
            assert targets is not None
            loss_cls, loss_reg = self.compute_loss(cls_out, box_out, anchors, targets)
            return {"loss_cls": loss_cls, "loss_reg": loss_reg}

        # inference: per ora ritorniamo i raw output
        return cls_out, box_out, anchors

    def compute_loss(self, cls_out, box_out, anchors, targets):
        device = cls_out.device
        batch_size, num_anchors, num_classes = cls_out.shape

        cls_targets = torch.zeros((batch_size, num_anchors, num_classes), device=device)
        box_targets = torch.zeros((batch_size, num_anchors, 4), device=device)
        box_mask = torch.zeros((batch_size, num_anchors), device=device)

        for b_idx in range(batch_size):
            gt_boxes = targets[b_idx]["boxes"].to(device)
            gt_labels = targets[b_idx]["labels"].to(device)

            if gt_boxes.numel() == 0:
                continue

            ious = box_iou(anchors.to(device), gt_boxes)  # (A,G)
            max_iou, max_ids = ious.max(dim=1)

            pos_mask = max_iou >= 0.5
            neg_mask = max_iou < 0.4
            ignore_mask = (~pos_mask) & (~neg_mask)

            # cls targets
            cls_targets[b_idx][neg_mask, :] = 0
            cls_targets[b_idx][ignore_mask, :] = -1
            cls_targets[b_idx][pos_mask, :] = 0

            pos_gt = gt_labels[max_ids[pos_mask]]
            cls_targets[b_idx][pos_mask, pos_gt] = 1

            # box targets
            box_mask[b_idx][pos_mask] = 1
            box_targets[b_idx][pos_mask] = encode_boxes(
                anchors[pos_mask].to(device),
                gt_boxes[max_ids[pos_mask]]
            )

        valid_mask = cls_targets != -1
        cls_inputs = cls_out[valid_mask]
        cls_tg = cls_targets[valid_mask]

        loss_cls = sigmoid_focal_loss(cls_inputs, cls_tg, alpha=0.25, gamma=2.0, reduction="sum")
        num_pos = box_mask.sum().clamp(min=1.0)
        loss_cls = loss_cls / num_pos

        pos_box = box_out[box_mask.bool()]
        pos_tg = box_targets[box_mask.bool()]
        loss_reg = smooth_l1_loss(pos_box, pos_tg, beta=1.0/9, reduction="sum") / num_pos

        return loss_cls, loss_reg

In [8]:
import torchvision.transforms.functional as TF  # 👈 NOTA: alias TF, non F

class ComposeTransforms:
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, image, target):
        for t in self.transforms:
            image, target = t(image, target)
        return image, target


class ToTensor:
    def __call__(self, image, target):
        image = TF.to_tensor(image)  # 👈 usiamo TF
        return image, target

In [33]:
def train_sasi(num_epochs,global_step,model,train_loader,device,optimizer,lr_scheduler):
        for epoch in range(num_epochs):
            model.train()
            running_loss = 0.0
    
            print("Inizio epoca")
            for images, targets in tqdm(train_loader, desc=f"Epoch {epoch}"):
                images = [img.to(device) for img in images]
                for t in targets:
                    t["boxes"] = t["boxes"].to(device)
                    t["labels"] = t["labels"].to(device)
    
                losses = model(images, targets)
                loss = losses["loss_cls"] + losses["loss_reg"]
    
                optimizer.zero_grad()
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
    
                running_loss += loss.item()
                global_step += 1
    
            lr_scheduler.step()
            avg_loss = running_loss / len(train_loader)
            print(f"Epoch {epoch}: loss={avg_loss:.4f}")
    
        # babysitting: salva checkpoint
        #torch.save(model.state_dict(), f"./retinanet_epoch_{epoch}.pth")

In [30]:
"""RIDUZIONE DATASETS PER TEST"""
def daima(mini_size,dataset_train,dataset_val):
    cartella = []
    # 1. Definisci quanto grande vuoi che sia il tuo mini-set (es. 50 immagini)
    mini_size = mini_size
    
    # 2. Crea una lista di indici (da 0 a 49)
    # Puoi anche usare indici random se preferisci un campione casuale
    indices = list(range(mini_size))

    if dataset_train != None:
        # 3. Crea il Subset
        mini_dataset_train = Subset(dataset_train, indices)
        
        # 4. Crea il DataLoader specifico per il mini dataset
        mini_loader_train = torch.utils.data.DataLoader(
            mini_dataset_train, 
            batch_size=4,           # Stesso batch size o diverso, come preferisci
            shuffle=True,           # Shuffle è utile anche nel test per verificare che il modello non crashi
            num_workers=2, 
            collate_fn=collate_fn   # Importante: mantieni la tua collate_fn
        )
        
        print(f"Dimensione dataset TRAIN originale: {len(dataset_train)}")
        print(f"Dimensione mini TRAIN dataset: {len(mini_dataset_train)}")

        cartella.append(mini_loader_train)
        

    if dataset_val != None:
        mini_dataset_val = Subset(dataset_val, indices)
        # 4. Crea il DataLoader specifico per il mini dataset
        mini_loader_val = torch.utils.data.DataLoader(
            mini_dataset_val, 
            batch_size=4,           # Stesso batch size o diverso, come preferisci
            shuffle=True,           # Shuffle è utile anche nel test per verificare che il modello non crashi
            num_workers=2, 
            collate_fn=collate_fn   # Importante: mantieni la tua collate_fn
        )
        
        print(f"Dimensione dataset VAL originale: {len(dataset_train)}")
        print(f"Dimensione mini VAL dataset: {len(mini_dataset_train)}")
        cartella.append(mini_loader_val)
    return cartella

In [ ]:
# train.py
import torch
from torch.utils.data import DataLoader
from torchvision.transforms import transforms
from tqdm import tqdm


def collate_fn(batch):
    """
    batch: lista di (image, target)
    image: tensor [C,H,W] (già dopo ToTensor)
    """
    images, targets = list(zip(*batch))

    # trova altezza e larghezza massima nel batch
    max_h = max(img.shape[1] for img in images)
    max_w = max(img.shape[2] for img in images)

    padded_images = []
    for img in images:
        c, h, w = img.shape
        # crea immagine "vuota" (zero = nero) della size massima
        pad = torch.zeros((c, max_h, max_w), dtype=img.dtype, device=img.device)
        # copia l'immagine originale in alto a sinistra
        pad[:, :h, :w] = img
        padded_images.append(pad)

    # targets li lasciamo così come sono (i box non cambiano)
    return padded_images, list(targets)


def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    num_classes = 91  # COCO categories (incl. background handling via one-hot)
    model = RetinaNet(num_classes=num_classes).to(device)

    train_transforms = ComposeTransforms([
        ToTensor(),
        # puoi aggiungere qui random resize, flip, ecc.
    ])

    train_dataset = COCODetectionRetinaNet(
        img_folder="/kaggle/input/cococustom/kaggle/working/train",
        ann_file="/kaggle/input/cococustom/kaggle/working/instances_train2017.json",
        transforms=train_transforms
    )

    train_loader = DataLoader(
        train_dataset,
        batch_size=2,
        shuffle=True,
        num_workers=0,      # tienilo 0 finché debuggiamo
        collate_fn=collate_fn
    )

    train_loader = daima(50,train_dataset,None)[0]


    optimizer = torch.optim.SGD(
        model.parameters(),
        lr=0.01,
        momentum=0.9,
        weight_decay=1e-4
    )
    lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(
        optimizer, milestones=[60, 80], gamma=0.1
    )

    num_epochs = 90
    global_step = 0
    train_sasi(num_epochs,global_step,model,train_loader,device,optimizer,lr_scheduler)

if __name__ == "__main__":
    main()

loading annotations into memory...
Done (t=20.85s)
creating index...
index created!
Using 11829 valid images out of original 118287
Dimensione dataset TRAIN originale: 11829
Dimensione mini TRAIN dataset: 50
Inizio epoca


Epoch 0: 100%|██████████| 13/13 [00:05<00:00,  2.49it/s]


Epoch 0: loss=1.6633
Inizio epoca


Epoch 1: 100%|██████████| 13/13 [00:04<00:00,  2.64it/s]


Epoch 1: loss=1.6987
Inizio epoca


...

Epoch 77: 100%|██████████| 13/13 [00:04<00:00,  2.70it/s]


Epoch 77: loss=1.3903
Inizio epoca


Epoch 78: 100%|██████████| 13/13 [00:04<00:00,  2.73it/s]


Epoch 78: loss=1.3621
Inizio epoca


Epoch 79:  69%|██████▉   | 9/13 [00:03<00:01,  2.73it/s]